In [3]:
import os
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes, hmac
import base64

from cryptography.hazmat.primitives import padding, serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa
import cryptography.hazmat.primitives.asymmetric as asymm

# encryption method AES-CBC-256
def Myencrypt(message, key, h):
    
    #key length check
    if len(key)<32:
        return "Error: This place is full of land mines, dragons, and dinosaurs with laser guns. Increase you key length to upgrade your armor."
    
    try:
        message = message.encode()
    except:
        pass
    
    iv = os.urandom(16)   #generate an iv
    if len(message)%16 != 0:
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(message) + padder.finalize()
        message = padded_data
    
    #HMAC
    #calling the default AES CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    #creating an encryptor object
    encryptor = cipher.encryptor()
    #generating cipher text
    ct = encryptor.update(message) + encryptor.finalize()
    #return (ct, iv) 
    
    tag = hmac.HMAC(h, hashes.SHA256(), backend=default_backend())
    tag.update(ct)
    tag = tag.finalize()
    return (ct, iv, tag)
    


def Mydecrypt(ct, iv, tag, key, h):
    #print(tag)
    #print(ct)
    
    #Verify
    h = hmac.HMAC(h, hashes.SHA256(), backend=default_backend())
    h.update(ct)
    h = h.finalize()
    
    if h == tag:
        print("verified")
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        #creating a decryptor object
        decryptor = cipher.decryptor()
        pt = decryptor.update(ct) + decryptor.finalize()
        try:
            unpadder = padding.PKCS7(128).unpadder()
            pt = unpadder.update(pt) + unpadder.finalize()
            return pt
        except:
            return pt
    else:
        return("invalid")



# execution code    
key = os.urandom(32)
h = os.urandom(32)
m = "a secret message"
# m = "hi"
c, iv, tag = Myencrypt(m, key, h)
print("Encrypted w/ tag")
print(c, iv, tag)

print("\ndecrypted")
Mydecrypt(c, iv, tag, key, h)


Encrypted w/ tag
b'v+\xc5\x11\xff6\x90s{\xad(\xdf\x97\xb1\x13\x85' b'\xf8\x93\xc0Y*\xe9\x8f\xe5e\xcd\xf5OC-\x14]' b'/yy.}\xed\xd0\xcdiB\x058\x1e\xbdqPf\xbe\x940\x99w\x83\x89/\x0ecQ\xec-\x04\xec'

decrypted
verified


b'a secret message'

In [6]:
# file encryption algorithm
def MyfileEncrypt(filepath):
    key = os.urandom(32)
    h = os.urandom(32)
    # Read the entire file as a single byte string
    with open(filepath, 'rb') as f:
        data = f.read()

    result = Myencrypt(data, key, h)
    ext = os.path.splitext(file_path)[1]
    result += (key, h, ext)
    
    image_result = open("encrypted_image.png", 'wb') # create a writable image and write the decoding result
    image_result.write(result[0])
    
    return result

# file dencryption algorithm
def MyfileDecrypt(enc_filepath, iv, tag, key, h, ext):
    
    with open(enc_filepath, 'rb') as f:
        data = f.read()
    
    file_name = "decrypted_image" + ext
    plaintext = Mydecrypt(data, iv, tag, key, h)
    image_result = open(file_name, 'wb') # create a writable image and write the decoding result
    image_result.write(plaintext)


# ----------------------
#Execution code
file_path = os.path.abspath("hello.png")
#ct, iv, tag, key, h, ext = MyfileEncrypt(file_path)
#MyfileDecrypt("encrypted_image.png", iv, tag, key, h, ext)

In [7]:
#create public/private key pair
def create_pem_key_pair():
    # create key object
    backend = default_backend()
    key = rsa.generate_private_key(backend=backend, public_exponent=65537,key_size=2048)
    
    # private key
    private_key = key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
            )
    with open("private.pem", 'wb') as private_pem:
        private_pem.write(private_key)
        private_pem.close()
    
    #public key
    public_key = key.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
    with open("public.pem", 'wb') as public_pem:
        public_pem.write(public_key)
        public_pem.close()
    
    
#RSA encryption method
def MyRSAEncrypt(file_path, RSA_Publickey_filepath):
    #encrypting an image file
    ct, iv, tag, key, h, ext = MyfileEncrypt(file_path)
    
    with open(RSA_Publickey_filepath, "rb") as p_key:
        public_key = serialization.load_pem_public_key(p_key.read(),backend=default_backend())
        
    #obtain RSACipher
    RSACipher = public_key.encrypt(key, asymm.padding.OAEP(
                                           mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),
                                           algorithm=hashes.SHA256(),
                                           label=None ))
    return RSACipher, ct, iv, tag, h, ext


#RSA decryption method
def MyRSADecrypt(RSACipher, ct, iv, tag, h, ext, RSA_Privatekey_filepath):

    with open(RSA_Privatekey_filepath, "rb") as key:
        private_key = serialization.load_pem_private_key(key.read(),password=None, backend=default_backend())

    key = private_key.decrypt(
        RSACipher,
        asymm.padding.OAEP(mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

    MyfileDecrypt(ct, iv, tag, key, h, ext)
    

#execution code
create_pem_key_pair() 
RSACipher, ct, iv, tag, h, ext = MyRSAEncrypt("hello.png", "public.pem")
MyRSADecrypt(RSACipher, "encrypted_image.png", iv, tag, h, ext, "private.pem")

verified
